# Optimization

In [4]:
import pandas as pd
import os 

In [6]:
# Read the distances matrix
df_matrix = pd.read_csv('../data/df_matrix.csv', index_col="ori")
df_matrix.head(3)

,Estació del Nord,Falla Almirante Cadarso-Conde de Altea,Falla Arzobispo Olaechea-San Marcelino,Falla Conde de Salvatierra-Cirilo Amorós-Mercat de Colón,Falla Convento Jerusalén-Matemático Marzal,Falla Cuba-Literato Azorín,Falla Císcar-Burriana,Falla Duc de Gaeta-Pobla de Farnals,Falla Espartero-Ramón y Cajal,Falla Exposición-Misser Mascó,...,Falla Plaza del Mercado Central,Falla Plaza del Pilar,Falla Quart-Extramurs-Velázquez,Falla Quart-Palomar,Falla Regne de Valencia-Duque de Calabria,Falla Reina-Paz-San Vicente (Falla Tio Pep),Falla Ribera-Convento Santa Clara,Falla San Vicente-Periodista Azzati,Falla Santa Genoveva Torres-Arquitecte Tolsà (la Nova de Orriols),Falla Sueca-Literato Azorín
ori,,,,,,,,,,,,,,,,,,,,,
Estació del Nord,0.0,2221.8,4124.1,1390.6,574.9,3100.0,2457.8,4504.7,1249.7,3922.9,...,1439.6,836.4,3081.4,1518.8,2272.8,942.0,359.1,642.1,4866.0,3176.9
Falla Almirante Cadarso-Conde de Altea,1085.7,0.0,5863.4,413.3,1632.2,1932.6,422.7,2507.1,2335.4,1925.3,...,2525.3,1922.1,4167.0,2604.4,667.5,2027.6,1444.8,1727.7,4597.0,2009.5
Falla Arzobispo Olaechea-San Marcelino,4034.5,4625.3,0.0,4814.5,3560.3,3633.2,4861.4,6457.8,3786.9,7445.6,...,4730.8,4127.6,4411.4,4810.0,4676.4,4392.3,3809.4,4092.4,8815.9,3710.1


In [7]:
# Read the routes df
df_routes = pd.read_csv('../data/df_routes.csv')
df_routes

,ori,des,osrm_dist,geometry
0,Falla Convento Jerusalén-Matemático Marzal,Falla Cuba-Literato Azorín,3154.6,"[[-0.379878, 39.466528], [-0.379727, 39.466493..."
1,Falla Convento Jerusalén-Matemático Marzal,Falla Sueca-Literato Azorín,3231.5,"[[-0.379878, 39.466528], [-0.379727, 39.466493..."
2,Falla Convento Jerusalén-Matemático Marzal,Falla Almirante Cadarso-Conde de Altea,2276.4,"[[-0.379878, 39.466528], [-0.379727, 39.466493..."
3,Falla Convento Jerusalén-Matemático Marzal,Falla Na Jordana,2410.0,"[[-0.379878, 39.466528], [-0.379727, 39.466493..."
4,Falla Convento Jerusalén-Matemático Marzal,Falla Plaza del Pilar,1120.1,"[[-0.379878, 39.466528], [-0.379727, 39.466493..."
...,...,...,...,...
865,Falla Santa Genoveva Torres-Arquitecte Tolsà (...,Falla Arzobispo Olaechea-San Marcelino,8625.0,"[[-0.366265, 39.492586], [-0.366185, 39.492525..."
866,Falla Santa Genoveva Torres-Arquitecte Tolsà (...,Falla Justo Vilar-Mercado del Cabañal,5475.1,"[[-0.366265, 39.492586], [-0.366185, 39.492525..."
867,Falla Santa Genoveva Torres-Arquitecte Tolsà (...,Falla Islas Canarias-Trafalgar-Samuel Ros,4850.7,"[[-0.366265, 39.492586], [-0.366185, 39.492525..."
868,Falla Santa Genoveva Torres-Arquitecte Tolsà (...,Falla Ribera-Convento Santa Clara,5343.8,"[[-0.366265, 39.492586], [-0.366185, 39.492525..."


In [8]:
"""Simple Travelling Salesperson Problem (TSP) between cities."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = df_matrix.round(0).astype(int).values.tolist()
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data


def print_solution(manager, routing, solution):
    sol = []
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()} miles")
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} ->"
        sol.append(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += f" {manager.IndexToNode(index)}\n"
    print(plan_output)
    plan_output += f"Route distance: {route_distance}meters\n"

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
            
        routes.append(route)
    return routes



def optimize_tsp():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)
        return get_routes(solution, routing, manager)

In [9]:
sol = optimize_tsp()[0]

Objective: 44233 miles
Route for vehicle 0:
 0 -> 22 -> 26 -> 21 -> 19 -> 24 -> 15 -> 20 -> 28 -> 13 -> 2 -> 5 -> 30 -> 25 -> 6 -> 16 -> 1 -> 3 -> 11 -> 9 -> 7 -> 12 -> 14 -> 17 -> 29 -> 10 -> 18 -> 23 -> 8 -> 4 -> 27 -> 0



In [10]:
# optimized_sequence 
optimized_sequence = list(df_matrix.index[sol])
optimized_sequence = [[optimized_sequence[i],optimized_sequence[i+1]]  for i in range(len(optimized_sequence)-1)]
optimized_sequence

[['Estació del Nord', 'Falla Plaza del Pilar'],
 ['Falla Plaza del Pilar', 'Falla Reina-Paz-San Vicente (Falla Tio Pep)'],
 ['Falla Reina-Paz-San Vicente (Falla Tio Pep)',
  'Falla Plaza del Mercado Central'],
 ['Falla Plaza del Mercado Central', 'Falla Na Jordana'],
 ['Falla Na Jordana', 'Falla Quart-Palomar'],
 ['Falla Quart-Palomar', 'Falla Llanterna-Na Rovella-Avda. de l’Oest'],
 ['Falla Llanterna-Na Rovella-Avda. de l’Oest', 'Falla Plaza de la Merced'],
 ['Falla Plaza de la Merced', 'Falla San Vicente-Periodista Azzati'],
 ['Falla San Vicente-Periodista Azzati',
  'Falla Jerónima Galés-Litógrafo Pascual y Abad'],
 ['Falla Jerónima Galés-Litógrafo Pascual y Abad',
  'Falla Arzobispo Olaechea-San Marcelino'],
 ['Falla Arzobispo Olaechea-San Marcelino', 'Falla Cuba-Literato Azorín'],
 ['Falla Cuba-Literato Azorín', 'Falla Sueca-Literato Azorín'],
 ['Falla Sueca-Literato Azorín', 'Falla Regne de Valencia-Duque de Calabria'],
 ['Falla Regne de Valencia-Duque de Calabria', 'Falla Císcar

In [11]:
df_routes_opt = pd.DataFrame(columns=df_routes.columns)
for seq in optimized_sequence:
    df_line = df_routes.loc[df_routes.ori.eq(seq[0]) & df_routes.des.eq(seq[1])]
    df_routes_opt = pd.concat([df_routes_opt,df_line])

df_routes_opt.reset_index(drop=True, inplace=True)
df_routes_opt

/var/folders/8c/_kn82sm11xg_1xzyn7clzt1w0000gn/T/ipykernel_96951/927058643.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_routes_opt = pd.concat([df_routes_opt,df_line])


,ori,des,osrm_dist,geometry
0,Falla Plaza del Pilar,Falla Reina-Paz-San Vicente (Falla Tio Pep),1270.2,"[[-0.383211, 39.471669], [-0.38311, 39.471683]..."
1,Falla Reina-Paz-San Vicente (Falla Tio Pep),Falla Plaza del Mercado Central,540.7,"[[-0.378759, 39.472202], [-0.378787, 39.472165..."
2,Falla Plaza del Mercado Central,Falla Na Jordana,1311.7,"[[-0.381235, 39.473774], [-0.381272, 39.473774..."
3,Falla Na Jordana,Falla Quart-Palomar,1242.1,"[[-0.38001, 39.480319], [-0.379891, 39.479779]..."
4,Falla Quart-Palomar,Falla Llanterna-Na Rovella-Avda. de l’Oest,813.1,"[[-0.387383, 39.475231], [-0.387917, 39.475135..."
5,Falla Llanterna-Na Rovella-Avda. de l’Oest,Falla Plaza de la Merced,39.0,"[[-0.383154, 39.472534], [-0.382703, 39.472571]]"
6,Falla Plaza de la Merced,Falla San Vicente-Periodista Azzati,619.2,"[[-0.382703, 39.472571], [-0.38234, 39.472601]..."
7,Falla San Vicente-Periodista Azzati,Falla Jerónima Galés-Litógrafo Pascual y Abad,2543.8,"[[-0.380499, 39.469614], [-0.380673, 39.469371..."
8,Falla Jerónima Galés-Litógrafo Pascual y Abad,Falla Arzobispo Olaechea-San Marcelino,1368.3,"[[-0.394822, 39.451006], [-0.394844, 39.451], ..."
9,Falla Arzobispo Olaechea-San Marcelino,Falla Cuba-Literato Azorín,3633.2,"[[-0.393309, 39.443346], [-0.392851, 39.44324]..."


In [ ]:
# df_routes_opt.to_csv('data/df_routes_opt.csv', index=False)